<a href="https://colab.research.google.com/github/atloveu1226/CMIT2024/blob/main/script2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Some important packages that are necessary in this project:

In [141]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.optimize import linprog
import time

With some parameters setting intially:

In [160]:
b   = [3, 1, 2, 2, 2, 0, 1, 1, 0, 2, 1, 2, 1, 0, 0]

Aeq = np.array([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
       [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]])

beq = np.array(b[:8])

c = [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]



lb = np.zeros(15)

ub = np.ones(15)

We define a nerual network, which we feed a 15-dimensional vector and output a 15-dimensional vector.

In [71]:
obs_size = 15
n_actions = 15  #it has 15 different kind of actions
HIDDEN_SIZE = 120

net= nn.Sequential(
            nn.Linear(obs_size, HIDDEN_SIZE),
            nn.ReLU(),
            #nn.Sigmoid(),
            nn.Linear(HIDDEN_SIZE, n_actions)
        )

objective = nn.CrossEntropyLoss() #quite standard for classification tasks
optimizer = optim.Adam(params=net.parameters(), lr=0.001)

It also define a 'state' function, to numerically to compare the result when we change the coefficient vector $𝐜$. The result shows that whether the diagonal sum of corresponding binary matrix satisfies the given condition.

In [102]:
def state(x):

  Dia_ob = b[8:15]

  #result = np.zeros(len(Dia_ob))

  count = 0

  i = 0

  #Translate diagonal restriction into matrix

  M = np.array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

  # Get the temporary variable

  tem = np.dot(M, x)

  for i in range(len(Dia_ob)):

    if tem[i] == Dia_ob[i]:

      count = count + 1

  #state[count - 1] = 1

  return count

We also need to define 'action' function, exerting action on $𝐜$.

In [103]:
sm = nn.Softmax(dim=1) #Softmax converts the 15-dimensional output vector to a probability distribution
def select_action(c):

  c_tensor = torch.FloatTensor([c])

  act_probs_t = sm(net(c_tensor))

  #print(act_probs_t)

  act_probs = act_probs_t.data.numpy()[0]

  action = np.random.choice(len(act_probs), p=act_probs) #chooses randomly one of the 4 actions according to the probabilities returned by the net

  return action

Example for the last function:

In [104]:
print('c is', select_action(c))

c is 5


To simplify the code, we write a new function for 'linprog'.

In [105]:
def object(c):

  result = linprog(
    c,
    A_eq = Aeq,
    b_eq = beq,
    bounds = list(zip(lb, ub)),
    method = 'highs',
   )

  return result

In [82]:
#print(object(c).x, state(object(c).x))

We then define the 'next_action' function:

In [154]:
def next_step(c, action):

  flag = False

  reward = 0

  result = c.copy()

  #action = select_action(c)

  #print(action)

  #print(c)

  result[action] = 1 - c[action]

  #para.x is an array, para.success return a bool value

  para1 = object(c)

  para2 = object(result)

  if para1.success == True and para2.success == True:

    if state(para1.x) < state(para2.x):

      reward = 1

      flag = True

    else:

      reward = 0

      flag = False

  #if state(para2.x) == 7 or para2.success == False:

    #flag = True

  #else:

    #flag = False



  return result, reward, flag

In [155]:
action = select_action(c)

print(next_step(c, action))

([1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], 0, False)


Preparation for training.

In [156]:
BATCH_SIZE = 200 #200

GAMMA = 0.9

PERCENTILE = 30 #30
REWARD_GOAL = 3.0

from collections import namedtuple  #more readable tuples
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])

Do the training process:

In [161]:
start_time = time.time()

iter_no = 0
reward_mean = 0
full_batch = []
batch = []
episode_steps = []
episode_reward = 0.0
stat_e = c

while reward_mean < REWARD_GOAL:
        action = select_action(stat_e)
        next_state, reward, episode_is_done = next_step(stat_e, action)


        episode_steps.append(EpisodeStep(observation=stat_e, action=action))
        episode_reward += reward

        #print(episode_steps)
        #print(episode_reward)

        if episode_is_done: # Episode finished
            batch.append(Episode(reward=episode_reward, steps=episode_steps))


            #print(len(batch))

            next_state = c
            episode_steps = []
            episode_reward = 0.0

            if len(batch) == BATCH_SIZE: # New set of batches ready --> select "elite"

                #print("Batch full")
                #print(batch)
                #print("\n")
                #input("Press Enter to continue...")


                reward_mean = float(np.mean(list(map(lambda s: s.reward, batch)))) #compute mean reward (lambda is inline function)
                elite_candidates= batch
                #elite_candidates= batch + full_batch
                returnG = list(map(lambda s: s.reward * (GAMMA ** len(s.steps)), elite_candidates))
                reward_bound = np.percentile(returnG, PERCENTILE) #lowest score that is greater than PERCENTILE% of scores in the data set
                                                                  #Keep the highest 100-PERCENTILE %
                #print("Batch finished", returnG, reward_bound)
                #input("Press Enter to continue...")

                train_obs = []
                train_act = []
                elite_batch = []

                for example, discounted_reward in zip(elite_candidates, returnG):
                        if discounted_reward > reward_bound:
                        #if discounted_reward >= reward_bound:
                              train_obs.extend(map(lambda step: step.observation, example.steps))
                              train_act.extend(map(lambda step: step.action, example.steps))
                              elite_batch.append(example)
                full_batch=elite_batch
                stat_e=train_obs
                acts=train_act

                #print(stat_e)
                #print(acts)
                #input("Press Enter to continue...")

                #Do the training
                if len(full_batch) != 0 : # just in case empty during an iteration
                  state_t = torch.FloatTensor(stat_e) #batch of states: [[1.0,0,0,0,0,0,0,0,0,0],[1,...]]
                  acts_t = torch.LongTensor(acts) # batch of actions: [0,2,3,1,..]

                  #print(state_t)
                  #print(acts_t)
                  #input("Press Enter to continue...")
                  optimizer.zero_grad() #it is good practice to do this, initializing the gradient computations
                  action_scores_t = net(state_t)

                  #print(action_scores_t)
                  #input("Press Enter to continue...")

                  loss_t = objective(action_scores_t, acts_t)
                  loss_t.backward() #computes the gradients
                  optimizer.step() #updates the weights according to the gradients
                  print("%d: loss=%.3f, reward_mean=%.3f" % (iter_no, loss_t.item(), reward_mean))
                  iter_no += 1
                batch = [] #empty the batch
        stat_e = next_state
        print(next_state, action)

print("--- %s seconds ---" % (time.time() - start_time))

流式输出内容被截断，只能显示最后 5000 行内容。
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 

KeyboardInterrupt: 

In [153]:
done=0
counter=0
MAXCOUNTER=15
stat_e = c

print("Step: "+str(counter))
print(stat_e)

while done !=1 and counter<MAXCOUNTER:
  state_t = torch.FloatTensor([stat_e])
  action_scores_t = net(state_t)
  act_probs_t = sm(action_scores_t)
  #print(act_probs_t)
  act_probs = act_probs_t.data.numpy()[0]
  proposed_action=np.argmax(act_probs)
  next_state, reward, done= next_step(stat_e, proposed_action)
  #stat_e=next_state.tolist() #changes NumPy array to a Python list
  counter+=1
  print("Step: "+str(counter))
  print(stat_e)

Step: 0
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
Step: 1
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
Step: 2
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
Step: 3
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
Step: 4
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
Step: 5
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
Step: 6
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
Step: 7
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
Step: 8
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
Step: 9
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
Step: 10
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
Step: 11
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
Step: 12
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
Step: 13
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
Step: 14
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
Step: 15
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
